In [10]:
import pandas as pd
import geopandas as gpd

In [11]:
from pathlib import Path
import re

files = []

for path in Path('imagens').rglob('*.*'):
    for i in re.findall(r"([A-Z]{2,4}-[0-9]{2})", path.name):
        files.append([i, path.as_posix()])
#     print(re.findall(r"([A-Z]{2,4}-[0-9]{2})", path.name), path )

In [12]:
df_imagens = pd.DataFrame(files, columns=['cod', 'path'])

In [13]:
df_cods = pd.DataFrame(df_imagens.groupby('cod')['path'].apply(list))

In [14]:
df_imagens = pd.DataFrame(list(map(lambda x: x, df_cods.path))).add_prefix('imagem_')

In [15]:
df_imagens['cod'] = df_cods.index

In [16]:
df_imagens.set_index('cod')

,imagem_0,imagem_1,imagem_2,imagem_3,imagem_4,imagem_5,imagem_6,imagem_7
cod,,,,,,,,
AUR-00,imagens/BOTUJURU/EQUIPAMENTOS NOVOS/AUR-00 Pra...,imagens/BOTUJURU/EQUIPAMENTOS NOVOS/AUR-00 Pra...,None,None,None,None,None,None
AUR-01,imagens/JORDANOPOLIS/AUR-01 Praça Domingos Van...,imagens/JORDANOPOLIS/AUR-01 Praça Domingos Van...,imagens/JORDANOPOLIS/AUR-01 Praça Domingos Van...,imagens/JORDANOPOLIS/AUR-01 Praça Domingos Van...,None,None,None,None
AUR-02,imagens/JORDANOPOLIS/AUR-02 Praça Ronald. De C...,imagens/JORDANOPOLIS/AUR-02 Praça Ronald. De C...,imagens/JORDANOPOLIS/AUR-02 Praça Ronald. De C...,imagens/JORDANOPOLIS/AUR-02 Praça Ronald. De C...,None,None,None,None
AUR-03,imagens/MONTANHÃO/AUR-03 Praça Henrique Trinda...,imagens/MONTANHÃO/AUR-03 Praça Henrique Trinda...,imagens/MONTANHÃO/AUR-03 Praça Henrique Trinda...,None,None,None,None,None
AUR-04,imagens/MONTANHÃO/AUR-04 Praça Vicente de Paul...,imagens/MONTANHÃO/AUR-04 Praça Vicente de Paul...,imagens/MONTANHÃO/AUR-04 Praça Vicente de Paul...,imagens/MONTANHÃO/AUR-04 Praça Vicente de Paul...,imagens/MONTANHÃO/AUR-04 Praça Vicente de Paul...,imagens/MONTANHÃO/AUR-04 Praça Vicente de Paul...,None,None
...,...,...,...,...,...,...,...,...
SAU-15,imagens/INDEPENDÊNCIA/EQUIPAMENTOS EXISTENTES/...,imagens/INDEPENDÊNCIA/EQUIPAMENTOS EXISTENTES/...,imagens/INDEPENDÊNCIA/EQUIPAMENTOS EXISTENTES/...,imagens/INDEPENDÊNCIA/EQUIPAMENTOS EXISTENTES/...,imagens/INDEPENDÊNCIA/EQUIPAMENTOS EXISTENTES/...,imagens/INDEPENDÊNCIA/EQUIPAMENTOS EXISTENTES/...,None,None
SAU-16,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,None,None
SAU-17,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,None,None


In [19]:
df_equip = pd.read_csv('./GIS/planilhas/equipamentos-XY.csv', delimiter=';', decimal=',')

In [20]:
df_equip

,BAIRRO,NOME,X,Y,TIPO,ENDEREÇO,COD DO LOCAL,PRIORIDADE,ESTADO DO EQUIPAMENTO,Video_0
0,BATISTINI,EMEB 1 OCTAVIO EDGARD DE OLIVEIRA,-46.610875,-23.771093,EDUCAÇÃO,R. João Saldanha 424,ED-31,NaN,EXISTENTE,videos/batistini/equipamentos/EMEB 1 OCTAVIO E...
1,BATISTINI,EMEB ANTONIO DOS SANTOS FARIAS,-46.599049,-23.775686,EDUCAÇÃO,"R. Irati, 11",ED-32,NaN,EXISTENTE,videos/batistini/equipamentos/EMEB 2 ANTONIO D...
2,BATISTINI,EMEB CLEIA MARIA TEURES DE SOUZA,-46.599302,-23.775548,EDUCAÇÃO,"R. Irati, 2",ED-33,NaN,EXISTENTE,videos/batistini/equipamentos/EMEB 3 CLEIA MAR...
3,BATISTINI,EMEB MANOEL DE BARROS,-46.596923,-23.776791,EDUCAÇÃO,"Rua Canada, 1",ED-34,NaN,EXISTENTE,videos/batistini/equipamentos/EMEB 4 MANOEL DE...
4,BATISTINI,EMEB PROFESSOR WALDEMAR CANCIANI,-46.590605,-23.769596,EDUCAÇÃO,"R. José Martins Fernandes, 86",ED-35,NaN,EXISTENTE,videos/batistini/equipamentos/EMEB 5 PROFESSOR...
...,...,...,...,...,...,...,...,...,...,...
94,RIO GRANDE,UBS RIACHO GRANDE,-46.529684,-23.779811,SAÚDE,"R. Santa Maria, 20",SAU-20,NaN,EXISTENTE,videos/rio_grande/equipamentos/UBS RIACHO GRAN...
95,RIO GRANDE,Rio Grande - E.I/ Fundamental / Integral EMEB ...,-46.529114,-23.779681,EDUCAÇÃO,"R. João D'Ângelo, 71 - Vila do Rio Grande, CEP...",ED-61,3.0,REFORMA,NaN
96,TABOÃO,Praça José Pino Cassetari,-46.599272,-23.656457,REQUALIFICAÇÃO DE ÁREAS URBANAS,"Rua Neuchatel , 270",AUR-12,3.0,REQUALIFICAÇÃO,NaN
97,TABOÃO,Taboão - E. Infantil/ Fundamental/ ESCOLA EST...,-46.605547,-23.660592,EDUCAÇÃO,"Av. do Taboão, 3550 - Vila Santa Luzia, CEP 09...",NaN,NaN,REFORMA,NaN


In [21]:
df_equip['cod'] = df_equip['COD DO LOCAL ']
# df_equip.columns

In [22]:
df_equip = pd.merge(df_equip, df_imagens.set_index('cod'), on='cod', how='left')

In [23]:
gdf_equip = gpd.GeoDataFrame(df_equip, geometry=gpd.points_from_xy(df_equip.X, df_equip.Y))

In [24]:
gdf_equip.drop(['X', 'Y', 'COD DO LOCAL ',  ], axis=1).to_file('GIS/equipamentos-com-fotos.gpkg', driver='GPKG')

In [25]:
for b in gdf_equip.BAIRRO.unique():
    b_formatado = b.lower().replace(" ", "_")
    gdf_equip.drop(['X', 'Y', 'COD DO LOCAL ',  ], axis=1).loc[gdf_equip.BAIRRO == b].to_file(f'GIS/equipamentos-com-fotos-{b_formatado}.gpkg', driver='GPKG')